In [1]:
from simworld.utils.video_recorder import VideoRecorder
from simworld.utils.vector import Vector
from simworld.communicator.communicator import Communicator
from simworld.communicator.unrealcv import UnrealCV
from simworld.agent.humanoid import Humanoid

import pandas as pd
import cv2
import os
import time

In [2]:
communicator = Communicator(UnrealCV())

INFO:__init__:234:Got connection confirm: b'connected to gym_citynav'


=>Info: using ip-port socket


In [3]:
# 设置保存路径
save_dir = r'C:\Users\28262\Desktop\PlayGorund\SimWorld\examples\dataset\scene_15'
os.makedirs(save_dir, exist_ok=True)

# 初始化数据记录
df = pd.DataFrame(columns=['step', 'action_index', 'action_type', 'image_path'])
step = 0

# 定义action类型描述
action_descriptions = [
    'humanoid_step_forward',
    'humanoid_stop'
] + ['humanoid_rotate'] * 12

# 方式1: 使用lambda函数，让action_list存储可调用的函数
action_list = [
    lambda: communicator.unrealcv.humanoid_step_forward('Base_User_Agent_C_0', 1),
    lambda: communicator.unrealcv.humanoid_stop('Base_User_Agent_C_0'),
] + [lambda: communicator.unrealcv.humanoid_rotate('Base_User_Agent_C_0', 30)] * 12

In [4]:
while step < 1000:
    # 获取当前要执行的动作函数
    action_index = step % len(action_list)
    action_func = action_list[action_index]
    action_type = action_descriptions[action_index]

    # 调用函数执行动作
    result = action_func()
    print(f"Step {step}: Executed {action_type} (index {action_index}), result: {result}")

    # 获取camera观察
    camera_image = communicator.get_camera_observation(0, 'lit', 'direct')

    # 保存图像
    image_filename = f"step_{step:06d}.png"
    image_path = os.path.join(save_dir, image_filename)
    cv2.imwrite(image_path, camera_image)

    # 记录到DataFrame
    new_row = {
        'step': step,
        'action_index': action_index,
        'action_type': action_type,
        'image_path': image_path
    }
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

    # 增加步骤计数
    step += 1

    # 添加短暂延时，避免执行过快
    time.sleep(0.1)

    # 每隔100步保存CSV并打印进度
    if step % 100 == 0:
        csv_path = os.path.join(save_dir, 'action_log.csv')
        df.to_csv(csv_path, index=False)
        print(f"Progress: {step}/1000 steps completed, CSV saved to {csv_path}")

# 最后保存完整的CSV
csv_path = os.path.join(save_dir, 'action_log.csv')
df.to_csv(csv_path, index=False)
print(f"All steps completed! Final CSV saved to {csv_path}")
print(f"Total images saved: {len(df)}")
print(f"Images saved in: {save_dir}")

Step 0: Executed humanoid_step_forward (index 0), result: None
Step 1: Executed humanoid_stop (index 1), result: None
Step 2: Executed humanoid_rotate (index 2), result: None
Step 3: Executed humanoid_rotate (index 3), result: None
Step 4: Executed humanoid_rotate (index 4), result: None
Step 5: Executed humanoid_rotate (index 5), result: None
Step 6: Executed humanoid_rotate (index 6), result: None
Step 7: Executed humanoid_rotate (index 7), result: None
Step 8: Executed humanoid_rotate (index 8), result: None
Step 9: Executed humanoid_rotate (index 9), result: None
Step 10: Executed humanoid_rotate (index 10), result: None
Step 11: Executed humanoid_rotate (index 11), result: None
Step 12: Executed humanoid_rotate (index 12), result: None
Step 13: Executed humanoid_rotate (index 13), result: None
Step 14: Executed humanoid_step_forward (index 0), result: None
Step 15: Executed humanoid_stop (index 1), result: None
Step 16: Executed humanoid_rotate (index 2), result: None
Step 17: Exe

KeyboardInterrupt: 